# Short introducton to MuPIF model API

* MuPIF comes with object-oriented design, where classes represent entities of certain kind (like models, properties, etc)
* Top level classes define common abstract interface that is common to all entities of certain kind
* These interfaces allow to manipulate different entities of the same kind using the same, generic interface

In MuPIF we have following fundamental classes:
* MuPIFObject class essentially defines methods allowing to get/set metadata to the entity. It is a base class for other classes (Workflow, Model, DataType classes). 
* Workflow class representing simulation workflow
* Model class representing individual modelling tasks (simulations)
* DataType classes representing individual data types supported by MuPIF (properties, fields, etc)

<table>
<tr><td width="50%">
<img src="Images/SampleWorkflowSchema.png" width="50%"> </td><td>

``` python
a=mupif.ConstantProperty(value=10.*mupif.U.deg_C, propID=mupif.DataID.PID_Temperature, ...)
b=mupif.constantProperty(value=10.*mupif.U.deg_C, propID=mupif.DataID.PID_Temperature, ...)
modelA.set(a, objectID="param_a")
modelA.set(b, objectID="param_b")
modelA.solveStep()

c=modelA.get(mupif.DataID.FID_Temperature)
model2.set(c, objectID="param_c")
model2.solveStep()
d=model2.get(mupif.DataID.FID_Displacement)
```
</td></tr></table>


## MuPIF Model class

This class defines a common interface to all models. It is derived from MuPIFObject class.

Model class interface:

| Method                                  |                                   Description |
| :- | :- |
| getMetadata(self, key, default=None)    | Returns metadata associated to given key. Inheritted from MuPIFObject |
| hasMetadata(self, key)                  | Returns true if key defined. Inheritted from MuPIFObject              |
| setMetadata(self, key, val)             | Sets metadata associated to given key. Inheritted from MuPIFObject    |
| initialize(self, workdir='', metadata=None, validateMetaData=True, **kwargs) | Initializes application, i.e. all functions after constructor and before run|
| set(self, obj, objectID="")             | Registers the given MuPIF data object in application |
| get(self, objectTypeID, time=None, objectID="") | Returns the MuPIF data object at given time. Object is identified by objectID |
| solveStep(self, tstep, stageID=0, runInBackground=False) | Solves (updates) the model for given time step |
| finishStep(self, tstep)                 | Called after a global convergence within a time step is achieved |
| getCriticalTimeStep(self)               | Returns a critical time step for an application |
| terminate(self)                         | Terminates the model |
| getApplicationSignature(self)           | Returns application signature |

For complete Model class definition please refer to https://github.com/mupif/mupif/blob/master/mupif/model.py

In addition, each model (represented by a class derived from Model class) has to define compulsory model metadata. These metadata are defined by schema. 



## Model metadata explained

Model metadata contain description of the model. The metadata structure and format is defined by JSON schema (see model.py for full reference). Compulsory entries in bold.

### Basic metadata (model name, id, version, description)
<pre>
    MD = {
        "<b>Name</b>": "Stationary thermal problem",
        "<b>ID</b>": "Thermo-1",
        "<b>Description</b>": "Stationary heat conduction using finite elements on rectangular domain",
        "<b>Version_date</b>": "1.0.0, Feb 2019",
        "Geometry": "2D rectangle",
        "Boundary_conditions": "Dirichlet, Neumann",
        ...
        }
</pre>
### Metadata describing physics behind the model
<pre>
    MD = {
        ...
        "<b>Physics</b>": {
            "<b>Type</b>": "Continuum",
            "<b>Entity</b>": "Finite volume",
            "Equation": ["Heat balance"],
            "Equation_quantities": ["Heat flow"],
            "Relation_description": ["Fick's first law"],
            "Relation_formulation": ["Flow induced by thermal gradient on isotropic material"],
            "Representation": "Finite volumes"
        },
        ...
    }
</pre>
### Metadata describing solver (how the equations are solved)
<pre>
    MD= {
        ...
        "<b>Solver</b>": {
            "<b>Software</b>": "own",
            "<b>Type</b>": "Finite elements",
            "<b>Accuracy</b>": "Medium",
            "<b>Sensitivity</b>": "Low",
            "<b>Complexity</b>": "Low",
            "<b>Robustness</b>": "High",
            "<b>Estim_time_step_s</b>": 1,
            "<b>Estim_comp_time_s</b>": 1.e-3,
            "<b>Estim_execution_cost_EUR</b>": 0.01,
            "<b>Estim_personnel_cost_EUR</b>": 0.01,
            "<b>Required_expertise</b>": "None",
            "<b>Language</b>": "Python",
            "<b>License</b>": "LGPL",
            "<b>Creator</b>": "Borek Patzak",
            "<b>Version_date</b>": "1.0.0, Feb 2019",
            "<b>Documentation</b>": "Felippa: Introduction to finite element methods, 2004",
        },
    }
</pre>
### Metadata defining input and output parameters of the model
<pre>
    MD = {
        "<b>Inputs</b>": [
        {
            "<b>Name</b>": "edge temperature",
            "<b>Type</b>": "mupif.Property",
            "<b>Required</b>": False,
            "<b>Type_ID</b>": "mupif.DataID.PID_Temperature",
            "<b>Units</b>": "degC",
            "Obj_ID": [
                "Cauchy top",
                "Cauchy bottom",
                "Cauchy left",
                "Cauchy right",
                "Dirichlet top",
                "Dirichlet bottom",
                "Dirichlet left",
                "Dirichlet right"
            ],
            "Set_at": "timestep",
            "<b>ValueType</b>": "Scalar"
        },
        {
            "<b>Name</b>": "Input file",
            "<b>Type</b>": "mupif.PyroFile",
            "<b>Required</b>": True,
            "<b>Type_ID</b>": "mupif.DataID.ID_InputFile",
            "Obj_ID": "input_file_thermal",
            "Set_at": "initialization",
            "<b>Units</b>": "none"
        }
    ],
    "<b>Outputs</b>": [
        {
            "<b>Name</b>": "temperature",
            "<b>Type_ID</b>": "mupif.DataID.FID_Temperature",
            "<b>Type</b>": "mupif.Field",
            "<b>Required</b>": False,
            "<b>Units</b>": "degC"
        }
    ],
    }
</pre>


&copy; CTU Team, 2021.